In [ ]:
import music21
import pickle
import numpy as np
from music21 import duration

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

### Load data from files

https://verovio.humdrum.org/?file=essen/europa/deutschl/altdeu2/deut4207.krn

In [ ]:
data = []

In [ ]:
files = ["data_germany_1","data_germany_2","data_germany_3","data_germany_4","data_germany_5","data_germany_6","data_germany_7", "data_germany_8", "data_germany_9", "data_germany_10", "data_germany_11"]
for file in files:
    with open('../data/data_2/{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

### Extracting the data

In [ ]:
notes_input = []
durations_input = []
phrases_input = []

for piece in data:
    notes = piece[0]
    notes_input.append(129) # START = 129
    notes_input.extend(notes)

    durations = piece[1]
    durations_input.append(0)
    durations_input.extend(durations)

    phrases = piece[2]
    phrases_input.append(0)
    phrases_input.extend(phrases)

In [ ]:
phrases_length = dict()
counter = 0
for i in range(len(phrases_input)):
    counter+=1
    if phrases_input[i] == 1:
        # print(i)
        # print(phrases_input)
        # print(counter)
        if counter in phrases_length.keys():
            phrases_length[counter] += 1
        else:
            phrases_length[counter] = 1
        counter = 0
        

In [ ]:
keys = list(phrases_length.keys())
keys.sort()
sorted_dict = { key : phrases_length[key] for key in keys}
sorted_dict

In [ ]:
print("Length of the notes_input: {}".format(len(notes_input)))
print("Twenty first elements: {}".format(notes_input[:20]))
print("Length of the durations_input: {}".format(len(durations_input)))
print("Twenty first elements: {}".format(durations_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_input)))
print("Twenty first elements: {}".format(phrases_input[:20]))

In [ ]:
result = dict()
for i in range(len(notes_input)):
    if phrases_input[i] == 1:
        if notes_input[i] not in result.keys():
            result[notes_input[i]] = 1
        else:
            result[notes_input[i]] +=1
result

In [ ]:
for i in range(len(notes_input)):
    if phrases_input[i] == 1 and notes_input[i] == 128:
        phrases_input[i] = 0
        k = i - 1
        while k > 0:
            if notes_input[k] == 128:
                k -= 1
            else:
                phrases_input[k] = 1
                break

In [ ]:
result = dict()
for i in range(len(notes_input)):
    if phrases_input[i] == 1:
        if notes_input[i] not in result.keys():
            result[notes_input[i]] = 1
        else:
            result[notes_input[i]] +=1
result

In [ ]:
durations_to_int = {}
int_to_duration = {}
notes_to_int = {}
int_to_notes = {}
phrases_to_int = {}
int_to_phrases = {}
for index, duration in enumerate(sorted(set(durations_input))):
    durations_to_int[duration] = index
    int_to_duration[index] = duration

for index, note in enumerate(sorted(set(notes_input))):
    notes_to_int[note] = index
    int_to_notes[index] = note

for index, phrase in enumerate(sorted(set(phrases_input))):
    phrases_to_int[phrase] = index
    int_to_phrases[index] = phrase

durations_network_input = [ durations_to_int[duration] for duration in durations_input ]
notes_network_input = [ notes_to_int[note] for note in notes_input ]
phrases_network_input = [ phrases_to_int[phrase] for phrase in phrases_input ]

In [ ]:
int_to_duration

In [ ]:
print("Length of the notes_input: {}".format(len(notes_network_input)))
print("Twenty first elements: {}".format(notes_network_input[:20]))
print("Length of the durations_input: {}".format(len(durations_network_input)))
print("Twenty first elements: {}".format(durations_network_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_network_input)))
print("Twenty first elements: {}".format(phrases_network_input[:20]))

In [ ]:
n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [ ]:
print("Number of distinct notes: {}".format(n_notes))
print("Number of distinct durations: {}".format(n_durations))

In [ ]:
with open('dictionary.pkl', 'wb') as file:
    dictionary = { "durations_to_int" : durations_to_int, "int_to_duration" : int_to_duration, "notes_to_int" : notes_to_int, "int_to_notes" : int_to_notes, "phrases_to_int" : phrases_to_int, "int_to_phrases" : int_to_phrases }
    # A new file will be created
    pickle.dump(dictionary, file)

In [ ]:
import os
import pickle
import numpy
from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from RNNmodel import prepare_sequences, create_network

In [ ]:
seq_len = 8
network_input, network_output = prepare_sequences(notes_network_input, durations_network_input, phrases_network_input, seq_len)

In [ ]:
print('note input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('phrases_output')
print(network_output[0][0])

In [ ]:
embed_size = 100
rnn_units = 256
use_attention = True
model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)
model.summary()

In [ ]:
import numpy as np
weights_folder = "weights" #os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))


checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )